# ONNX Quick Start Guide

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/onnx/onnx/blob/main/examples/quickstart.ipynb)

This notebook demonstrates how to:
1. Create a simple PyTorch model
2. Export it to ONNX format
3. Validate the ONNX model
4. Run inference using ONNX Runtime
5. Visualize the model graph

**Total time: ~5 minutes**

## 1. Installation

Install the required packages:

In [1]:
# Install dependencies
!pip install -q onnx torch onnxruntime numpy

Der Befehl "pip" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


In [2]:
import torch
import torch.nn as nn
import onnx
import onnxruntime as ort
import numpy as np

print(f"PyTorch version: {torch.__version__}")
print(f"ONNX version: {onnx.__version__}")
print(f"ONNX Runtime version: {ort.__version__}")

ModuleNotFoundError: No module named 'torch'

## 2. Create a PyTorch Model

Let's create a simple convolutional neural network for image classification:

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc = nn.Linear(32 * 8 * 8, 10)
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = self.fc(x)
        return x

# Create model instance
model = SimpleCNN()
model.eval()

print("Model created successfully!")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")

## 3. Export PyTorch Model to ONNX

Now we'll export the model to ONNX format. ONNX uses a static graph, so we need to provide example inputs:

In [ ]:
# Create dummy input (batch_size=1, channels=3, height=32, width=32)
dummy_input = torch.randn(1, 3, 32, 32)

# Export to ONNX
onnx_model_path = "simple_cnn.onnx"

torch.onnx.export(
    model,                          # PyTorch model
    dummy_input,                    # Example input
    onnx_model_path,                # Output file path
    export_params=True,             # Store trained parameters
    opset_version=17,               # ONNX opset version
    do_constant_folding=True,       # Optimize constant folding
    input_names=['input'],          # Input tensor name
    output_names=['output'],        # Output tensor name
    dynamic_axes={
        'input': {0: 'batch_size'},  # Variable batch size
        'output': {0: 'batch_size'}
    }
)

print(f"✅ Model exported to {onnx_model_path}")

## 4. Validate the ONNX Model

Let's verify that the exported model is valid:

In [ ]:
# Load the ONNX model
onnx_model = onnx.load(onnx_model_path)

# Check the model
try:
    onnx.checker.check_model(onnx_model)
    print("✅ ONNX model is valid!")
except onnx.checker.ValidationError as e:
    print(f"❌ Model validation failed: {e}")

# Print model information
print(f"\nModel IR version: {onnx_model.ir_version}")
print(f"Opset version: {onnx_model.opset_import[0].version}")
print(f"Producer: {onnx_model.producer_name}")
print(f"\nNumber of nodes: {len(onnx_model.graph.node)}")

## 5. Inspect Model Graph

Let's examine the model's inputs, outputs, and operators:

In [ ]:
# Print inputs
print("Model Inputs:")
for input_tensor in onnx_model.graph.input:
    print(f"  - {input_tensor.name}: {[d.dim_value if d.dim_value != 0 else 'dynamic' for d in input_tensor.type.tensor_type.shape.dim]}")

# Print outputs
print("\nModel Outputs:")
for output_tensor in onnx_model.graph.output:
    print(f"  - {output_tensor.name}: {[d.dim_value if d.dim_value != 0 else 'dynamic' for d in output_tensor.type.tensor_type.shape.dim]}")

# Count operators
from collections import Counter
op_types = Counter([node.op_type for node in onnx_model.graph.node])
print("\nOperator Types:")
for op_type, count in op_types.most_common():
    print(f"  - {op_type}: {count}")

## 6. Run Inference with ONNX Runtime

Now let's run inference using the ONNX Runtime for optimized performance:

In [ ]:
# Create ONNX Runtime session
ort_session = ort.InferenceSession(onnx_model_path)

# Get input name
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name

print(f"Input name: {input_name}")
print(f"Output name: {output_name}")

# Prepare input data
test_input = np.random.randn(1, 3, 32, 32).astype(np.float32)

# Run inference
ort_outputs = ort_session.run([output_name], {input_name: test_input})

print(f"\n✅ Inference completed!")
print(f"Output shape: {ort_outputs[0].shape}")
print(f"Predicted class: {np.argmax(ort_outputs[0])}")

## 7. Compare PyTorch vs ONNX Runtime

Let's verify that both produce the same results:

In [ ]:
# PyTorch inference
with torch.no_grad():
    torch_input = torch.from_numpy(test_input)
    torch_output = model(torch_input).numpy()

# ONNX Runtime inference
onnx_output = ort_outputs[0]

# Compare outputs
print("PyTorch output:")
print(torch_output[0][:5])  # First 5 values

print("\nONNX Runtime output:")
print(onnx_output[0][:5])  # First 5 values

# Calculate difference
max_diff = np.abs(torch_output - onnx_output).max()
print(f"\nMaximum difference: {max_diff}")

if max_diff < 1e-5:
    print("✅ Outputs match! The models are numerically equivalent.")
else:
    print("⚠️ Outputs differ - this might be expected due to floating point precision.")

## 8. Benchmark Performance (Optional)

Let's compare inference speed between PyTorch and ONNX Runtime:

In [ ]:
import time

num_runs = 100
batch_input = np.random.randn(10, 3, 32, 32).astype(np.float32)

# Benchmark PyTorch
torch_input_batch = torch.from_numpy(batch_input)
start = time.time()
with torch.no_grad():
    for _ in range(num_runs):
        _ = model(torch_input_batch)
torch_time = (time.time() - start) / num_runs

# Benchmark ONNX Runtime
start = time.time()
for _ in range(num_runs):
    _ = ort_session.run([output_name], {input_name: batch_input})
onnx_time = (time.time() - start) / num_runs

print(f"PyTorch average time: {torch_time*1000:.2f} ms")
print(f"ONNX Runtime average time: {onnx_time*1000:.2f} ms")
print(f"Speedup: {torch_time/onnx_time:.2f}x")

## 9. Visualize the Model (Optional)

You can visualize the ONNX model graph using Netron:

In [ ]:
# In Colab or Jupyter, you can visualize with netron
try:
    import netron
    print("Netron is installed. You can visualize the model at:")
    print("https://netron.app/")
    print("Or run: netron simple_cnn.onnx")
except ImportError:
    print("To visualize the model, install netron:")
    print("  pip install netron")
    print("Then run: netron simple_cnn.onnx")
    print("\nOr upload your model to https://netron.app/")

## Summary

In this quick start, you learned:

✅ How to export a PyTorch model to ONNX format  
✅ How to validate ONNX models  
✅ How to inspect model structure and operators  
✅ How to run inference with ONNX Runtime  
✅ How to verify numerical equivalence  
✅ How to benchmark performance  

## Next Steps

- 📖 [ONNX Documentation](https://onnx.ai/onnx/)
- 🔧 [PyTorch ONNX Export Guide](https://pytorch.org/docs/stable/onnx.html)
- 🚀 [ONNX Runtime Documentation](https://onnxruntime.ai/)
- 💡 [ONNX Model Zoo](https://github.com/onnx/models) - Pre-trained ONNX models
- 🛠️ [ONNX Tutorials](https://github.com/onnx/tutorials) - Advanced examples

## Questions or Issues?

- 💬 [ONNX GitHub Discussions](https://github.com/onnx/onnx/discussions)
- 🐛 [Report Issues](https://github.com/onnx/onnx/issues)
- 📧 [Community Slack](https://lfaifoundation.slack.com/)